<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/MFC/MFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MFC

## Mounting Gdrive

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports

In [16]:
import pandas as pd

## Functions

In [17]:
# https://stackoverflow.com/questions/55695479/why-cant-i-sort-a-list-of-dicts-in-3-x-why-do-i-get-typeerror-not-suppor
# https://www.trainingint.com/how-to-find-duplicates-in-a-python-list.html#:~:text=list%20contains%20duplicates-,mylist%20%3D%20%5B5%2C%203%2C%205%2C%202%2C%201,5%20%26%206%20are%20duplicate%20numbers.&text=Alternatively%2C%20we%20can%20create%20a,to%20alert%20us%20of%20duplicates.&text=return%20False-,mylist%20%3D%20%5B5%2C%203%2C%205%2C%202%2C%201,repeated%20number%20in%20the%20list.

from operator import itemgetter
def return_duplicates(mylist=[]):
    duplist = [] # empty list to hold the duplicate elements from the list
    sorted_list = sorted(mylist, key=itemgetter('start'))
    for index, item in enumerate(sorted_list):
        # print(item)
        try:
            if(item['code'] == sorted_list[index+1]['code'] 
               and (item['start'] == sorted_list[index+1]['start'] or item['start']+1 == sorted_list[index+1]['start']) 
               and (item['end'] == sorted_list[index+1]['end'] or item['end']+1 == sorted_list[index+1]['end'])
               ):
                if item not in duplist:
                    duplist.append(item)
        except IndexError:
            pass      
    return duplist

def adding_duplicate_annotationsDF(df):

    workingDF = df[['annotations','headline_frame','primary_frame', 'text']].copy()
    workingDF.insert(loc=4,  column='duplicates_annotations', value="")

    for index, x in enumerate(tab_df['annotations']):
        annotator = x['framing']
        dupli = []
        # print('r', annotator)
        if len(annotator) != 0:
            for a in annotator:
                # print(a)
                for c in x['framing'][a]:
                    # print(c)
                    dupli.append(c)
            # print(return_duplicates(dupli))
            workingDF.iat[index, 4]= return_duplicates(dupli)

    return workingDF

    

## DATASETS

### Immigration

In [19]:
IMM_df = pd.read_json(f'{dir}mfc_v4.0/immigration/immigration_labeled.json')
IMM_df = IMM_df.T
IMM_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
Immigration1.0-10005,"{'framing': {'hannah_47.0_r': [{'code': 10.2, ...",10.1,0,10.2,17,IMM-10005\n\nPRIMARY\n\nImmigrants without HOP...,2001,8,15,atlanta journal and constitution
Immigration1.0-10006,"{'framing': {'alex_54.0_r': [{'code': 5.2, 'en...",15.1,0,5.2,19,"IMM-10006\n\nPRIMARY\n\nReaction to Tancredo, ...",2001,8,15,denver post
Immigration1.0-10013,"{'framing': {'jeremy_44.0_r': [{'code': 2.2, '...",1.1,0,2.2,19,"IMM-10013\n\nPRIMARY\n\n$50,000 per entry\n\nT...",2001,8,14,herald-sun
Immigration1.0-10015,"{'framing': {'corey_55.0_r': [{'code': 10.2, '...",None,0,10.2,17,IMM-10015\n\nPRIMARY\n\nDangerous crossing; PU...,2001,8,13,atlanta journal and constitution
Immigration1.0-10018,"{'framing': {'haley_46.0_r': [{'code': 11.2, '...",None,0,None,None,IMM-10018\n\nPRIMARY\n\nCHILDREN OF LATINO IMM...,2001,8,13,san jose mercury news
...,...,...,...,...,...,...,...,...,...,...
Immigration1.0-9981,"{'framing': {'allie_54.0_r': [{'code': 13.2, '...",13.1,0,13.2,17,IMM-9981\n\nPRIMARY\n\nBush Goes Slow on Immig...,2001,8,20,washington post
Immigration1.0-9983,"{'framing': {'haley_49.0_r': [{'code': 2.2, 'e...",None,0,2.2,19,IMM-9983\n\nPRIMARY\n\nTHE OPEN FORUM\n\nLAMM ...,2001,8,19,denver post
Immigration1.0-9986,"{'framing': {'corey_43.0_r': [{'code': 12.2, '...",None,0,12.2,19,IMM-9986\n\nPRIMARY\n\nImmigration Foes Find P...,2001,8,19,washington post
Immigration1.0-9987,"{'framing': {'corey_54.0_r': [{'code': 13.2, '...",None,0,13.2,17,IMM-9987\n\nPRIMARY\n\nEasy immigration finds ...,2001,8,18,atlanta journal and constitution


In [20]:
IMM_df[['annotations','text']].iloc[0:2]

,annotations,text
Immigration1.0-10005,"{'framing': {'hannah_47.0_r': [{'code': 10.2, ...",IMM-10005\n\nPRIMARY\n\nImmigrants without HOP...
Immigration1.0-10006,"{'framing': {'alex_54.0_r': [{'code': 5.2, 'en...","IMM-10006\n\nPRIMARY\n\nReaction to Tancredo, ..."


In [21]:
IMM_df['annotations'].iloc[0]

{'framing': {'hannah_47.0_r': [{'code': 10.2, 'end': 18, 'start': 11},
   {'code': 10.1, 'end': 70, 'start': 20},
   {'code': 10.0, 'end': 257, 'start': 72},
   {'code': 11.0, 'end': 573, 'start': 417},
   {'code': 5.0, 'end': 785, 'start': 628},
   {'code': 11.0, 'end': 953, 'start': 787},
   {'code': 5.0, 'end': 1010, 'start': 955},
   {'code': 10.0, 'end': 1355, 'start': 1115},
   {'code': 11.0, 'end': 1545, 'start': 1486}],
  'kylie_47.0_r': [{'code': 10.2, 'end': 18, 'start': 11},
   {'code': 10.1, 'end': 70, 'start': 20},
   {'code': 10.0, 'end': 416, 'start': 72},
   {'code': 10.0, 'end': 786, 'start': 575},
   {'code': 5.0, 'end': 1011, 'start': 955},
   {'code': 10.0, 'end': 1356, 'start': 954}]},
 'irrelevant': {'hannah_47.0_r': False, 'kylie_47.0_r': False},
 'tone': {'corey_58.0_rt': [{'code': 17.35, 'end': 18, 'start': 11}],
  'spencer_58.2_rt': [{'code': 17.35, 'end': 18, 'start': 11}]}}

### Gun Control

In [22]:
GC_df = pd.read_json(f'{dir}mfc_v4.0/guncontrol/guncontrol_labeled.json')
GC_df = GC_df.T
GC_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
gun_control1.0-10,"{'framing': {}, 'irrelevant': {'hannah_122.0_r...",None,1,None,None,GUN-10\n\nPRIMARY\n\nNJ argues before NJ Sup C...,1975,5,13,new york times
gun_control1.0-10007,"{'framing': {'marc_134.0_r': [{'code': 13.2, '...",7.1,0,13.2,19,"GUN-10007\n\nPRIMARY\n\nCHANGE THE LAWS, PROTE...",2005,12,21,daily news
gun_control1.0-10008,"{'framing': {'gareth_117.0_r': [{'code': 7.2, ...",6.1,0,7.2,19,GUN-10008\n\nPRIMARY\n\nTIME FOR ALBANY TO GET...,2005,12,18,daily news
gun_control1.0-10013,"{'framing': {'acacia_133.0_r': [{'code': 5.2, ...",5.1,0,5.2,18,GUN-10013\n\nPRIMARY\n\nMan found not guilty i...,2005,12,16,denver post
gun_control1.0-10014,"{'framing': {'allie_134.0_r': [{'code': 13.2, ...",7.1,0,13.2,18,GUN-10014\n\nPRIMARY\n\nPolice Deaths And Stal...,2005,12,15,new york times
...,...,...,...,...,...,...,...,...,...,...
gun_control1.0-9985,"{'framing': {'allie_117.0_r': [{'code': 5.2, '...",3.1,0,5.2,19,GUN-9985\n\nPRIMARY\n\nCHURCH WINS ROUND IN GU...,2004,1,14,saint paul pioneer press
gun_control1.0-9988,"{'framing': {'griffin_130.0_r': [{'code': 7.2,...",7.1,0,7.2,None,GUN-9988\n\nPRIMARY\n\nReport Details Crime Gu...,2004,1,13,washington post
gun_control1.0-9989,{'framing': {'brandon_133.0_r': [{'code': 13.2...,6.1,0,13.2,17,"GUN-9989\n\nPRIMARY\n\nOn Gun Control, Dean Ai...",2004,1,9,washington post
gun_control1.0-9990,"{'framing': {'jade_129.0_r': [{'code': 5.2, 'e...",7.1,0,5.2,17,GUN-9990\n\nPRIMARY\n\nEX-SHERIFF'S AIDE FIGHT...,2004,1,8,saint paul pioneer press


### Death Penalty

In [23]:
DP_df = pd.read_json(f'{dir}mfc_v4.0/deathpenalty/deathpenalty_labeled.json')
DP_df = DP_df.T
DP_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
death_penalty_100,{'framing': {'sabrina_167.recode_r': [{'code':...,13.1,0,13.2,None,DEA-100\n\nPRIMARY\n\nJERSEY'S ASSEMBLY APPROV...,1983,6,24,new york times
death_penalty_10005,"{'framing': {}, 'irrelevant': {'marc_163.2_r':...",None,1,None,None,DEA-10005\n\nPRIMARY\n\nDEATH SENTENCE IN DEPU...,1997,6,14,san jose mercury news
death_penalty_10006,"{'framing': {'marc_171.2_r': [{'code': 10.2, '...",7.1,0,10.2,17,DEA-10006\n\nPRIMARY\n\n2 MEN ORDERED TO DIE S...,1997,6,14,san jose mercury news
death_penalty_10013,{'framing': {'jade_168.recode_r': [{'code': 8....,8.1,0,8.2,17,DEA-10013\n\nPRIMARY\n\nREPRISAL HINT AT MCVEI...,1997,6,13,daily news
death_penalty_10017,{'framing': {'jade_163.recode_r': [{'code': 5....,None,0,5.2,None,"DEA-10017\n\nPRIMARY\n\n'Cruel, unusual' uncle...",1997,6,13,denver post
...,...,...,...,...,...,...,...,...,...,...
death_penalty_9964,{'framing': {'priscilla_168.recode_r': [{'code...,15.1,0,4.2,19,DEA-9964\n\nPRIMARY\n\nJune 8-14; Open-Handed ...,1997,6,15,new york times
death_penalty_9967,{'framing': {'kaleigh_170.recode_r': [{'code':...,5.1,0,5.2,18,DEA-9967\n\nPRIMARY\n\nDEATH OR LIFE: PENALTY ...,1997,6,15,philadelphia inquirer
death_penalty_9969,"{'framing': {}, 'irrelevant': {'sabrina_167.re...",None,1,None,None,DEA-9969\n\nPRIMARY\n\nAS THEY SEE IT\n\n'It's...,1997,6,15,san jose mercury news
death_penalty_9975,"{'framing': {}, 'irrelevant': {}, 'tone': {'ma...",None,None,None,18,DEA-9975\n\nPRIMARY\n\nTIMOTHY McVEIGH SENTENC...,1997,6,14,atlanta journal and constitution


### Same Sex

In [24]:
SS_df = pd.read_json(f'{dir}mfc_v4.0/samesex/samesex_labeled.json')
SS_df = SS_df.T
SS_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
same-sex_marriage1.0-10,"{'framing': {}, 'irrelevant': {}, 'tone': {'kr...",None,None,None,17,SSM-10\n\nPRIMARY\n\nDEAD MAN'S LAWYERS PURSUE...,2002,11,10,st. louis post-dispatch
same-sex_marriage1.0-100,"{'framing': {}, 'irrelevant': {}, 'tone': {}}",None,None,None,None,SSM-100\n\nPRIMARY\n\nSTATE HIGH COURT REJECTS...,2002,1,4,san jose mercury news
same-sex_marriage1.0-1000,"{'framing': {}, 'irrelevant': {'claire_86.0_r'...",None,1,None,19,SSM-1000\n\nPRIMARY\n\nMinister preaches about...,2004,2,14,atlanta journal and constitution
same-sex_marriage1.0-10000,"{'framing': {}, 'irrelevant': {}, 'tone': {'da...",None,None,None,18,SSM-10000\n\nPRIMARY\n\nThe Nation; Civil Righ...,1996,9,15,new york times
same-sex_marriage1.0-10001,"{'framing': {}, 'irrelevant': {}, 'tone': {'al...",None,None,None,19,SSM-10001\n\nPRIMARY\n\nAtlanta Sued Over Its ...,1996,9,15,new york times
...,...,...,...,...,...,...,...,...,...,...
same-sex_marriage1.0-9995,"{'framing': {'claire_86.0_r': [{'code': 12.2, ...",6.1,0,12.2,19,SSM-9995\n\nPRIMARY\n\nSAME-SEX PAIRS DEALT A ...,1996,9,19,san jose mercury news
same-sex_marriage1.0-9996,"{'framing': {}, 'irrelevant': {}, 'tone': {'ha...",None,None,None,17,SSM-9996\n\nPRIMARY\n\nDenver Extends Health C...,1996,9,18,new york times
same-sex_marriage1.0-9997,"{'framing': {}, 'irrelevant': {}, 'tone': {'al...",None,None,None,17,SSM-9997\n\nPRIMARY\n\n...Voting One's Conscie...,1996,9,17,washington post
same-sex_marriage1.0-9998,"{'framing': {}, 'irrelevant': {}, 'tone': {'da...",None,None,None,19,SSM-9998\n\nPRIMARY\n\nUptoSpeed; THE WEEK'S T...,1996,9,15,atlanta journal and constitution


### Tobacco

In [29]:
tab_df = pd.read_json(f'{dir}mfc_v4.0/tobacco/tobacco_labeled.json')
tab_df = tab_df.T
tab_df

,annotations,headline_frame,irrelevant,primary_frame,primary_tone,text,year,month,day,source
Tobacco1.0-100,"{'framing': {'laura_99.0_r': [{'code': 1.2, 'e...",14.1,0,1.2,18,TOB-100\n\nPRIMARY\n\nTobacco firms find hospi...,1990,10,16,usa today
Tobacco1.0-10003,"{'framing': {}, 'irrelevant': {}, 'tone': {'ha...",None,None,None,19,TOB-10003\n\nPRIMARY\n\nTOBACCO RULE SOMETHING...,1996,6,2,st. louis post-dispatch
Tobacco1.0-10004,"{'framing': {}, 'irrelevant': {'gareth_94.0_r'...",None,1,None,None,TOB-10004\n\nPRIMARY\n\nAround the South; Regi...,1996,6,1,atlanta journal and constitution
Tobacco1.0-10005,"{'framing': {'jeremy_99.0_r': [{'code': 3.2, '...",3.1,0,3.2,17,TOB-10005\n\nPRIMARY\n\nTobacco supporters pra...,1996,6,1,herald-sun
Tobacco1.0-10012,"{'framing': {'hannah_97.0_r': [{'code': 9.2, '...",3.1,0,9.2,19,TOB-10012\n\nPRIMARY\n\nBRIEFING\n\nMESA TO FL...,1995,12,29,denver post
...,...,...,...,...,...,...,...,...,...,...
Tobacco1.0-9971,"{'framing': {}, 'irrelevant': {'jeremy_92.0_r'...",None,1,None,None,TOB-9971\n\nPRIMARY\n\nHEALTH WATCH; Study tar...,1996,6,11,atlanta journal and constitution
Tobacco1.0-9973,"{'framing': {'dana_93.0_r': [{'code': 4.2, 'en...",11.1,0,4.2,17,TOB-9973\n\nPRIMARY\n\nSmoking is becoming a s...,1996,6,11,denver post
Tobacco1.0-9977,"{'framing': {'gareth_93.0_r': [{'code': 5.2, '...",15.1,0,5.2,17,TOB-9977\n\nPRIMARY\n\n'Ashes to Ashes'\n\nTo ...,1996,6,9,new york times
Tobacco1.0-9979,"{'framing': {'corey_98.0_r': [{'code': 11.2, '...",15.1,0,11.2,19,TOB-9979\n\nPRIMARY\n\nSMOKE SCREEN IS SEEN BE...,1996,6,9,st. louis post-dispatch


In [46]:
tab_dup_df = adding_duplicate_annotationsDF(tab_df)
tab_dup_df[['text','duplicates_annotations']].iloc[:1]
# tab_dup_df

,text,duplicates_annotations
Tobacco1.0-100,TOB-100\n\nPRIMARY\n\nTobacco firms find hospi...,"[{'code': 1.2, 'end': 16, 'start': 9}, {'code'..."


In [51]:
st = tab_dup_df.iloc[,3]
st

"TOB-100\n\nPRIMARY\n\nTobacco firms find hospitality abroad\n\nU.S. tobacco interests are keeping a low profile in the ongoing ''sin tax'' battle, preferring to pay a little more in the USA while making a lot more in profit overseas.\n\nWith Congress struggling for ways to cut the deficit - including a possible 50% tax increase on tobacco - lawmakers have less time to consider bills that would restrict lucrative tobacco sales abroad.\n\n''You're talking millions and millions in gains (overseas) compared to four cents a pack (in taxes) here,'' says Tom Slane of the Tobacco Merchants Association.\n\nThe cigarette tax, now 16 cents a pack, would rise 4 cents in both 1991 and 1993 under the latest budget plan.\n\nIf higher taxes are approved, analysts predict most smokers will switch to less expensive, generic cigarettes.\n\nWorldwide, U.S. tobacco exports hit $ 4.1 billion in 1988. They rose at least $ 1 billion in 1989, and are expected to rise $ 2 billion in 1990.\n\nThe tobacco indust

In [71]:
# char
l = []
for index, x in enumerate(st):
    cod = ""
    if index >= 57 and index <227:
        # cod += x
        l.append(x)
        
print(l)
    

['U', '.', 'S', '.', ' ', 't', 'o', 'b', 'a', 'c', 'c', 'o', ' ', 'i', 'n', 't', 'e', 'r', 'e', 's', 't', 's', ' ', 'a', 'r', 'e', ' ', 'k', 'e', 'e', 'p', 'i', 'n', 'g', ' ', 'a', ' ', 'l', 'o', 'w', ' ', 'p', 'r', 'o', 'f', 'i', 'l', 'e', ' ', 'i', 'n', ' ', 't', 'h', 'e', ' ', 'o', 'n', 'g', 'o', 'i', 'n', 'g', ' ', "'", "'", 's', 'i', 'n', ' ', 't', 'a', 'x', "'", "'", ' ', 'b', 'a', 't', 't', 'l', 'e', ',', ' ', 'p', 'r', 'e', 'f', 'e', 'r', 'r', 'i', 'n', 'g', ' ', 't', 'o', ' ', 'p', 'a', 'y', ' ', 'a', ' ', 'l', 'i', 't', 't', 'l', 'e', ' ', 'm', 'o', 'r', 'e', ' ', 'i', 'n', ' ', 't', 'h', 'e', ' ', 'U', 'S', 'A', ' ', 'w', 'h', 'i', 'l', 'e', ' ', 'm', 'a', 'k', 'i', 'n', 'g', ' ', 'a', ' ', 'l', 'o', 't', ' ', 'm', 'o', 'r', 'e', ' ', 'i', 'n', ' ', 'p', 'r', 'o', 'f', 'i', 't', ' ', 'o', 'v', 'e', 'r', 's', 'e', 'a', 's', '.']


In [ ]:
TOB-100

PRIMARY

Tobacco firms find hospitality abroad

U.S. tobacco interests are keeping a low profile in the ongoing ''sin tax'' battle, preferring to pay a little more in the USA while making a lot more in profit overseas.

With Congress struggling for ways to cut the deficit - including a possible 50% tax increase on tobacco - lawmakers have less time to consider bills that would restrict lucrative tobacco sales abroad.

''You're talking millions and millions in gains (overseas) compared to four cents a pack (in taxes) here,'' says Tom Slane of the Tobacco Merchants Association.

The cigarette tax, now 16 cents a pack, would rise 4 cents in both 1991 and 1993 under the latest budget plan.

If higher taxes are approved, analysts predict most smokers will switch to less expensive, generic cigarettes.

Worldwide, U.S. tobacco exports hit $ 4.1 billion in 1988. They rose at least $ 1 billion in 1989, and are expected to rise $ 2 billion in 1990.

The tobacco industry also hopes its low-key approach will stall anti- smoking initiatives.

Earlier this year, smoking was banned on almost all U.S. air flights; some municipalities passed anti-smoking ordinances and banned cigarette sales from vending machines.

''We've more than held our own in Congress this year,'' says Thomas Lauria of The Tobacco Institute. ''We faced over 160 (anti-smoking) bills ... and held the line on all of them.''


